In [1]:
import silq
silq.initialize('Antimony', select=[])
a = silq.config.antimony


Initialization complete


# Actual test

In [1]:
import unittest
from blinker import signal, Signal
import tempfile
import silq
from silq.tools.config import *
silq.initialize('Serwan')

Initializing imports


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


Initialization complete


c:\users\serwa_000\documents\github\silq\silq\__init__.py:95: UserWarning: 'properties' should be added to SilQ config
  warnings.warn("'properties' should be added to SilQ config")


In [2]:
%%javascript

Jupyter.keyboard_manager.command_shortcuts.add_shortcut('r', {
    help : 'run cell',
    help_index : 'zz',
    handler : function (event) {
        IPython.notebook.restart_run_all();
        return false;
    }}
);

<IPython.core.display.Javascript object>

In [3]:
class TestConfig(unittest.TestCase):
    def setUp(self):
        self.folder = tempfile.TemporaryDirectory()
        self.d = {
            'pulses': {
                'read': {'t_start': 0,
                         't_stop': 10}},
            'connections': ['connection1', 'connection2']}
        self.config = DictConfig('env1', folder=self.folder.name,
                                 config=self.d)

    def tearDown(self):
        self.folder.cleanup()

    def test_dicts_equal(self):
        self.assertTrue(self.dicts_equal(self.d, self.d))
        self.assertTrue(self.dicts_equal(self.d, self.config))
        self.assertTrue(self.dicts_equal(self.config, self.config))
        self.config.x = 2
        self.assertFalse(self.dicts_equal(self.d, self.config))

    def test_save_load_dir(self):
        self.config.save(folder=self.folder.name, save_as_dir=True)
        
        filepath = os.path.join(self.folder.name, '{}.json'.format(self.config.name))
        folderpath = os.path.join(self.folder.name, self.config.name)
        self.assertFalse(os.path.exists(filepath))
        self.assertTrue(os.path.isdir(folderpath))
        for filename in self.config.keys():
            filepath = os.path.join(folderpath, '{}.json'.format(filename))
            self.assertTrue(os.path.exists(filepath))
        
        config_loaded = DictConfig('env1', folder=self.folder.name)
        self.assertTrue(config_loaded.save_as_dir)
        self.assertTrue(self.dicts_equal(self.config, config_loaded))
        
        new_folder = tempfile.TemporaryDirectory()
        config_loaded.pulses.save_as_dir = True
        config_loaded.save(folder=new_folder.name)
        
        os.path.isdir(os.path.join(new_folder.name, 'pulses'))
        os.path.isfile(os.path.join(new_folder.name, 'pulses', 'read.json'))
        os.path.isfile(os.path.join(new_folder.name, 'connections.json'))
        
        config_loaded2 = DictConfig('env1', folder=new_folder.name)
        self.assertTrue(self.dicts_equal(self.config, config_loaded2))
        self.assertIsInstance(config_loaded2.connections, ListConfig)
        
        
    def test_save_load_file(self):
        self.config.save(folder=self.folder.name)
        
        filepath = os.path.join(self.folder.name, '{}.json'.format(self.config.name))
        folderpath = os.path.join(self.folder.name, self.config.name)
        self.assertTrue(os.path.exists(filepath))
        self.assertFalse(os.path.exists(folderpath))
        
        config_loaded = DictConfig('env1', folder=self.folder.name)
        self.assertFalse(config_loaded.save_as_dir)
        self.assertTrue(self.dicts_equal(self.config, config_loaded))

    def dicts_equal(self, d1, d2):
        d1_keys = list(d1.keys())
        for d2_key, d2_val in d2.items():
            if d2_key not in d1_keys:
                return False
            else:
                d1_keys.remove(d2_key)
                d1_val = d1[d2_key]
                if isinstance(d1_val, dict):
                    if not isinstance(d2_val, dict):
                        return False
                    if not self.dicts_equal(d1_val, d2_val):
                        return False
                elif not d1_val == d2_val:
                    return False
        return True

In [4]:
suite_config = unittest.TestLoader().loadTestsFromModule(TestConfig())
suite = unittest.TestSuite([suite_config])
unittest.TextTestRunner(verbosity=2, failfast=True).run(suite)

test_dicts_equal (__main__.TestConfig) ... ok
test_save_load_dir (__main__.TestConfig) ... ok
test_save_load_file (__main__.TestConfig) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.184s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>